# Extract metadata from PDF files using fine-tuned GPT3 language model

This notebook will demonstrate how we can extract Dublin Core style metadata about PDF documents, in this case doctoral theses from four Finnish universities (Åbo Akademi, University of Turku, University of Vaasa and Lappeenranta University of Technology), using only the raw text from the first few pages of the PDF.

The set of 192 documents will be split into two subsets (train: 149, test: 43). We will extract the text from around 5 pages of text, aiming for 500 to 700 words. The corresponding metadata, which has been exported from DSpace repositories of the universities, is represented in a simple textual "key: value" format, which should be easy enough for a language model to handle. The train set is used to create a data set which will then be used to fine-tune a GPT-3 Curie model. Subsequently the model can be used to generate similar metadata for unseen documents from the test set.

For this experiment, an OpenAI API access key is required. It can be generated after registering an user account (the same account can be used for e.g. ChatGPT). The API key has to be stored in an environment variable `OPENAI_API_KEY` before starting this notebook. The finetuning will cost around \\$2.50 USD and generating new metadata with the API also has a small cost, but currently every account gets a free \\$18 credit from OpenAI which is plenty for this experiment even with a few iterations.

This notebook depends on a few Python libraries, which are listed in `requirements.txt`. See the README for details.

## Test the connection and API key

Make sure it's possible to use the OpenAI API.

In [1]:
import openai
import os

# read the OpenAI API key from an environment variable
openai.api_key = os.environ['OPENAI_API_KEY']

# test the API connection by making a simple request
response = openai.Completion.create(model="text-curie-001", prompt="Say this is a test", temperature=0, max_tokens=7)
response

<OpenAIObject text_completion id=cmpl-6bmAgQqLkF71IJL0qqBELSTuq2P2h at 0x7fd50f537060> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is a test."
    }
  ],
  "created": 1674461326,
  "id": "cmpl-6bmAgQqLkF71IJL0qqBELSTuq2P2h",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}

## Prepare the data set

Extract metadata and PDF text

In [2]:
# Define some settings for the metadata extraction

import glob

MAXPAGES = 5  # how many pages of text to extract (maximum)
MARGIN = 2  # how many more pages to look at, in case we can't find text from the first ones
TEXT_MIN = 500  # how many words to aim for (minimum)
TEXT_LIMIT = 700  # upper limit on # of words

# files containing metadata about doctoral theses documents, exported from DSpace repositories
METADATAFILES = glob.glob("dspace/*-doctheses.xml")

# metadata fields we are interested in (corresponding to fields used in DSpace)
# syntax: "fieldname" or "fieldname/qualifier"
METADATAFIELDS = """
title
title/alternative
contributor/faculty
contributor/author
contributor/organization
contributor/opponent
contributor/supervisor
contributor/reviewer
publisher
date/issued
relation/issn
relation/isbn
relation/ispartofseries
relation/numberinseries
""".strip().split()

# identifiers of documents that will form the test set
# these have been selected to correspond with a demo application for the same purpose
TEST_SET_IDS = """
handle_10024_181378
handle_10024_181284
handle_10024_181280
handle_10024_181229
handle_10024_181227
handle_10024_181210
handle_10024_181206
handle_10024_181139
handle_10024_181073
handle_10024_181025
handle_10024_181001
handle_10024_163335
handle_10024_163304
handle_10024_163298
handle_10024_163277
handle_10024_163276
handle_10024_163263
handle_10024_163258
handle_10024_163257
handle_10024_163057
handle_10024_163056
handle_10024_162878
handle_10024_11364
handle_10024_11363
handle_10024_11348
handle_10024_11207
handle_10024_10928
handle_10024_10620
handle_10024_10614
handle_10024_10443
handle_10024_10432
handle_10024_10254
handle_10024_152922
handle_10024_152903
handle_10024_152904
handle_10024_152860
handle_10024_152862
handle_10024_152853
handle_10024_152854
handle_10024_152855
handle_10024_152852
handle_10024_152846
handle_10024_152836
""".strip().split()

In [3]:
#%%time

from lxml import etree
import requests
import os.path
from pypdf import PdfReader
import glob

# train set: document identifiers, text (x) and metadata (y)
train_ids = []
train_x = []
train_y = []

# test set: document identifiers, text (x) and metadata (y)
test_ids = []
test_x = []
test_y = []

def extract_metadata(doc_item):
    """extract the metadata as a list of (key, value) tuples from an etree element representing a document"""
    metadata = []
    for fldname in METADATAFIELDS:
        if '/' in fldname:
            fld, qualifier = fldname.split('/')
            for val in doc_item.findall(f"metadata[@element='{fld}'][@qualifier='{qualifier}']"):
                if fld == 'date':
                    metadata.append((fldname, val.text[:4]))  # only the year                
                else:
                    metadata.append((fldname, val.text))
        else:
            for val in doc_item.findall(f"metadata[@element='{fldname}'][@qualifier='']"):
                metadata.append((fldname, val.text))
    return metadata

def id_to_fn(identifier):
    """convert a URI identifier to a simpler string we can use as a filename for the PDF"""
    return 'docs/' + identifier.replace('https://', '').replace('/','_') + ".pdf"

def download(file_url, identifier):
    """download a PDF file, with the given identifier, from the given URL (unless this was done already)
    and return a path to the PDF file"""
    path = id_to_fn(identifier)
    if os.path.exists(path) and os.path.getsize(path) > 0:
        return path

    response = requests.get(file_url)
    with open(path, "wb") as f:
        f.write(response.content)
        print(f"wrote {file_url} as {path}")
    return path

def extract_text(fn):
    """extract and return the first few pages of text from the given PDF file"""
    reader = PdfReader(fn)
    texts = []
    extracted_pages = 0
    extracted_length = 0
    for idx, page in enumerate(reader.pages[:MAXPAGES + MARGIN]):
        text = page.extract_text()
        text_length = len(text.strip().split())        
        if extracted_length + text_length < TEXT_LIMIT:
            texts.append(text)
            extracted_length += text_length
            extracted_pages += 1
        else:
            print(f"skipping page {idx+1} of {fn}: text would become too long")
        if extracted_pages >= MAXPAGES or extracted_length >= TEXT_MIN:
            break
    return '\n'.join(texts)

def is_test_doc(identifier):
    """return True iff the given identifier belongs to the test set"""
    shortid = 'handle' + identifier.split('handle')[1].replace('/', '_')
    return shortid in TEST_SET_IDS    

# Read all the metadata files, extract the DSpace metadata, download the PDFs and extract text from them
# into the train_* and test_* lists
for fn in METADATAFILES:
    tree = etree.parse(fn)
    for item in tree.findall('item'):
        try:
            identifier = item.find("metadata[@element='identifier'][@qualifier='uri']").text
        except AttributeError:
            print("no identifier found, skipping")
            continue
        try:
            file_url = item.find('file').text
        except AttributeError:
            print(f"no file element found (id: {identifier}), skipping")
            continue
            print(f"skipping test document {identifier}")
            continue
        path = download(file_url, identifier)
        text = extract_text(path)
        metadata = extract_metadata(item)
        if is_test_doc(identifier):
            test_ids.append(identifier)
            test_x.append(text)
            test_y.append(metadata)
        else:
            train_ids.append(identifier)
            train_x.append(text)
            train_y.append(metadata)

print(f"train set size: {len(train_ids)}")
print(f"test set size: {len(test_ids)}")

skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13521.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13449.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13420.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13378.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13377.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13257.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13205.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13177.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13169.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13167.pdf: text would become too long
skipping page 5 of docs/osuva.uwasa.fi_handle_10024_13159.pdf: text would become too long
skipping p

## Fine-tuning

Prepare a fine-tuning dataset and use it to fine-tune a GPT3 model.

In [4]:
# prepare fine-tuning dataset
import json

PROMPT_SUFFIX = '\n\n###\n\n'
COMPLETION_STOP = '\n###'
TRAINFILE = 'fine-tune.jsonl'

def metadata_to_text(metadata):
    """convert the metadata tuple to text with key: value pairs"""
    return "\n".join([f"{fld}: {val}" for fld, val in metadata])

def create_sample(text, metadata):
    """create a fine-tuning sample from text and metadata about a single document"""
    return {'prompt': text + PROMPT_SUFFIX,
            'completion': metadata_to_text(metadata) + COMPLETION_STOP}

with open(TRAINFILE, 'w') as outf:
    for text, metadata in zip(train_x, train_y):
        sample = create_sample(text, metadata)
        print(json.dumps(sample), file=outf)

print(f"wrote fine-tuning data set into file {TRAINFILE}")

wrote fine-tuning data set into file fine-tune.jsonl


In [5]:
# Optional:
# Check that the fine-tuning data set is OK using the prepare_data tool.
# It will complain that all completions start with the same "title:" prefix, this can be ignored.
# NOTE: The command has to be interrupted by pressing the stop button in Jupyter.
!openai tools fine_tunes.prepare_data -f fine-tune.jsonl

Analyzing...

- Your file contains 149 prompt-completion pairs
- All prompts end with suffix `\n\n###\n\n`
- All completions start with prefix `title: `. Most of the time you should only add the output data into the completion, without any prefix
- All completions end with suffix `\n###`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Remove prefix `title: ` from all completions [Y/n]: ^C



In [6]:
# Perform the actual finetuning via the API. This can take a while, there can be a long queue.
!openai api fine_tunes.create -t fine-tune.jsonl -m curie

Upload progress: 100%|███████████████████████| 643k/643k [00:00<00:00, 554Mit/s]
Uploaded file from fine-tune.jsonl: file-qggpGW5qlkmO5Moe0DzGP4Hw
Created fine-tune: ft-iwX4ajAH4TaptYpJdXFSSOu2
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-01-23 10:18:08] Created fine-tune: ft-iwX4ajAH4TaptYpJdXFSSOu2

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-iwX4ajAH4TaptYpJdXFSSOu2



In [7]:
# reattach in case the stream gets interrupted
#!openai api fine_tunes.follow -i <insert finetuning job id here>
!openai api fine_tunes.follow -i ft-iwX4ajAH4TaptYpJdXFSSOu2

[2023-01-23 10:18:08] Created fine-tune: ft-iwX4ajAH4TaptYpJdXFSSOu2
[2023-01-23 10:22:29] Fine-tune costs $2.40
[2023-01-23 10:22:29] Fine-tune enqueued. Queue number: 1
[2023-01-23 10:22:30] Fine-tune is in the queue. Queue number: 0
[2023-01-23 10:22:38] Fine-tune started
[2023-01-23 10:24:36] Completed epoch 1/4
[2023-01-23 10:25:56] Completed epoch 2/4
[2023-01-23 10:27:16] Completed epoch 3/4
[2023-01-23 10:28:38] Completed epoch 4/4
[2023-01-23 10:29:01] Uploaded model: curie:ft-personal-2023-01-23-08-29-00
[2023-01-23 10:29:02] Uploaded result file: file-ZDZOAruJOzm3JPYQMfjIQmIu
[2023-01-23 10:29:02] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2023-01-23-08-29-00 -p <YOUR_PROMPT>


In [8]:
# copy the model name from above output and store it in a variable
model_name = "curie:ft-personal-2023-01-23-08-29-00"

## Test the fine-tuned model

Give the model some documents from the test set that it has never seen before and see what kind of metadata it can extract. Compare that to the manually created metadata of the same documents, extracted from DSpace systems.

In [9]:
def get_completions(text):
    response = openai.Completion.create(model=model_name,
                                    prompt=text + PROMPT_SUFFIX,
                                    temperature=0,  # no fooling around!
                                    max_tokens=500, # should be plenty
                                    stop=[COMPLETION_STOP])  # stop at ###
    return response['choices'][0]['text']

# test it with some sample documents from the test set
for idx in (3,8,13,18,23,28,33,38):
    identifier = test_ids[idx]
    text = test_x[idx]
    metadata = test_y[idx]
    print(identifier)
    print("---")
    print("DSpace metadata:")
    print(metadata_to_text(metadata))
    print("---")
    print("Generated metadata:")
    gen_metadata = get_completions(text).strip()
    print(gen_metadata)
    print()
    
    

https://osuva.uwasa.fi/handle/10024/11207
---
DSpace metadata:
title: How to apply technology, knowledge and operations decision models for strategically sustainable resource allocation?
title/alternative: Kuinka soveltaa teknologiaan, tietoon ja tuotantoon liittyvän päätöksenteon malleja strategisesti kestävään resurssien allokointiin?
contributor/faculty: fi=Tekniikan ja innovaatiojohtamisen yksikkö|en=School of Technology and Innovations|
contributor/author: Tilabi, Sara
contributor/organization: fi=Vaasan yliopisto|en=University of Vaasa|
publisher: Vaasan yliopisto
date/issued: 2020
relation/issn: 2323-9123
relation/issn: 0355-2667
relation/isbn: 978-952-476-915-0
relation/ispartofseries: Acta Wasaensia
relation/numberinseries: 445
---
Generated metadata:
title: How to apply technology, knowledge and operations decision models for strategically sustainable resource allocation?
title/alternative: Kuinka soveltaa teknologiaan, tietoon ja tuotantoon liittyvää päätöksen teon malleja s

title: Life cycle cost-driven design for additive manufacturing: the frontier to sustainable manufacturing in laser-based powder bed fusion
contributor/faculty: fi=School of Energy Systems|en=School of Energy Systems|
contributor/author: Nyamekye, Patricia
contributor/organization: Lappeenrannan-Lahden teknillinen yliopisto LUT
contributor/organization: Lappeenranta-Lahti University of Technology LUT
contributor/opponent: Hryha, Eduard
contributor/reviewer: Hryha, Eduard
contributor/reviewer: Mazzi, Anna
publisher: Lappeenranta-Lahti University of Technology LUT
date/issued: 2021
relation/issn: 1456-4491
relation/ispartofseries: Acta Universitatis Lappeenrantaensis

